In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

class PenguinModel:
    def __init__(self):
        self.model = None
        self.label_encoder_species = LabelEncoder()
        self.label_encoder_sex = LabelEncoder()
        self.is_fitted = False
        self.species_labels = None
        self.scaler = StandardScaler()

    def read_data(self, filepath):
        """
        Чтение данных из CSV файла и предобработка
        """
        data = pd.read_csv(filepath)
        # Оставляем только необходимые столбцы
        data = data[['Species', 'Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Sex', 'Island']]
        data.dropna(inplace=True)

        # Преобразование категориальных переменных
        data['Species'] = self.label_encoder_species.fit_transform(data['Species'])
        data['Sex'] = self.label_encoder_sex.fit_transform(data['Sex'])
        data = pd.get_dummies(data, columns=['Island'], drop_first=True)

        X = data.drop(columns='Species')
        y = data['Species']

        # Сохраняем информацию о метках классов (видов пингвинов)
        self.species_labels = self.label_encoder_species.classes_

        # Масштабируем данные
        X_scaled = self.scaler.fit_transform(X)

        return X_scaled, y

    def class_balance(self, y):
        """
        Подсчет и вывод баланса классов (видов пингвинов)
        """
        species_counts = Counter(y)
        print("Баланс классов (видов пингвинов):")
        for species, count in species_counts.items():
            print(f"Класс {self.species_labels[species]}: {count} записей")

    def fit(self, X, y):
        """
        Обучение модели логистической регрессии
        """
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model = LogisticRegression(max_iter=2000)  # Увеличиваем количество итераций
        self.model.fit(X_train, y_train)
        self.is_fitted = True

        # Оценка точности на тестовой выборке
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")
        return accuracy

    def predict(self, X):
        """
        Предсказание классов для новых данных
        """
        if not self.is_fitted:
            raise ValueError("The model must be fitted before predicting.")
        X_scaled = self.scaler.transform(X)  # Масштабируем данные перед предсказанием
        return self.model.predict(X_scaled)

In [6]:
# Функция демонстрации модели с визуализацией
def demonstrate_penguin_model(filepath):
    # Шаг 1. Инициализация модели и чтение данных
    model = PenguinModel()
    X, y = model.read_data(filepath)

    # Шаг 2. Вывод баланса классов
    model.class_balance(y)

    # Шаг 3. Обучение модели
    accuracy = model.fit(X, y)
    print(f"Точность модели: {accuracy:.2f}")

# Пример использования
filepath = '../data/raw/penguins_lter.csv'  # Убедитесь, что у вас есть этот файл в той же директории
demonstrate_penguin_model(filepath)

Баланс классов (видов пингвинов):
Класс Adelie Penguin (Pygoscelis adeliae): 146 записей
Класс Chinstrap penguin (Pygoscelis antarctica): 68 записей
Класс Gentoo penguin (Pygoscelis papua): 120 записей
Accuracy: 1.00
Точность модели: 1.00
